### code for segregating dataset into separate folders

In [1]:
import pandas as pd
import os
import glob
import cv2
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from sklearn.utils import shuffle
from keras.preprocessing import image
import keras

Using TensorFlow backend.


In [3]:
annotation_df = pd.read_csv("..//dataset//Data_Entry_2017.csv",index_col="Image_Index")

In [4]:
annotation_df.head()

,Finding_Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
Image_Index,,,,,,,,,,
00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143
00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171
00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143


In [5]:
### read the list of image path stored
image_1 = glob.glob("..//dataset//images_001//images//*.png")
image_2 = glob.glob("..//dataset//images_002//images//*.png")
image_path = image_1 + image_2
print(len(image_path))

14999


In [6]:
### dummy val data
image_path_val = image_1[:100]

In [7]:
#### generator working on padas dataframe and 

#param:
#    image_path: list of image path
#    info_df: dataframe with info abt labels and other stuffs 
#    batch_size: the batch size expected
#Return:
#    shuffled pair of image and target

def generator(image_path,info_df,batch_size):
    
    size = len(image_path)
    target_string = ['Atelectasis','Consolidation','Infiltration','Pneumothorax','Edema','Emphysema',
                 'Fibrosis','Effusion','Pneumonia','Pleural_Thickening','Cardiomegaly','Nodule','Mass','Hernia','No Finding']

    
    #image_path , target = shuffle (image_path,target)
    while 1:
        for offset in range(0,int(len(image_path)/batch_size)+0):
            train = []
            #target_string = []
            end = offset*batch_size+batch_size
            if end >len(image_path):
                end = len(image_path)
                
                
            path = image_path[0+offset*batch_size:end]
            targ = []
            for i in path:
                target = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                vari = i.split('\\') #['..//dataset//images_001//images', '00000001_000.png']
                vari_target_string =  info_df.loc[[vari[1]]]["Finding_Labels"][vari[1]] #'Cardiomegaly|Effusion'
                
                #prinint this in order to see the correct class
                #print(vari_target_string)
                
                target_split = vari_target_string.split('|')
                for i in target_split:
                    idx = target_string.index(i)
                    #print(idx)
                    target[idx] = 1
                    
                #print this in order to see the correct match of target
                #print(target)
                
                targ.append(target)
                
                
            #targ = target[0+offset*batch_size:end]
            for a,b in zip(path,targ):
                
                img = image.load_img(a, target_size=(224, 224))
                x = image.img_to_array(img)
                x = np.expand_dims(x, axis=0)
                img = preprocess_input(x)
                
                img = np.reshape(img,(224,224,3))
                
                if (type(img) !=bool) and (type(img) != None):
                        #print (img.shape)
                        train.append(img)
                        #target_string.append(str(b))# works for one hot encoding
                        #target_string.append(b)
                
            
            X_train = np.array(train)
            Y_train = np.array(targ)
            #print("shh",X_train.shape)
         
            yield shuffle(X_train,Y_train)
           


In [8]:
batch_size = 8
training_generator = generator(image_path,annotation_df,batch_size)
val_generator = generator(image_path_val,annotation_df,batch_size)


In [9]:
v_next = next(training_generator)

In [10]:
v_next[0].shape

(8, 224, 224, 3)

In [11]:
v_next[1][3]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0])

### seperable Dense network

multi label image classificaiton

In [12]:
### construct the dense net

In [13]:
import densenet_separable


In [14]:
### here we are using sigmoid for multi label classification
image_dim = (224, 224, 3)
model = densenet_separable.DenseNetImageNet121(input_shape=image_dim,weights=None,classes=15,activation = 'sigmoid')

Instructions for updating:
Colocations handled automatically by placer.
The activation used here is :  sigmoid


In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 112, 112, 64) 339         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         separable_conv2d_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

activation_119 (Activation)     (None, 7, 7, 992)    0           batch_normalization_119[0][0]    
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 7, 7, 128)    126976      activation_119[0][0]             
__________________________________________________________________________________________________
batch_normalization_120 (BatchN (None, 7, 7, 128)    512         conv2d_61[0][0]                  
__________________________________________________________________________________________________
activation_120 (Activation)     (None, 7, 7, 128)    0           batch_normalization_120[0][0]    
__________________________________________________________________________________________________
separable_conv2d_59 (SeparableC (None, 7, 7, 32)     5248        activation_120[0][0]             
__________________________________________________________________________________________________
concatenat

### load weight if needed

In [16]:
model.load_weights("weights\\dense_seperable_sigmoid_15\\weights.02-0.42.hdf5")

### optimizer choice

here we are using binary crossentropy as we see adding sigmoid to a multiple nuron output layers make this task into a binary(1-0) classificaiton for individual neurons . thus changing it from increasing only one probablity of a label (softmax) it becomes n number of binary problem

Refer
https://www.analyticsvidhya.com/blog/2019/04/build-first-multi-label-image-classification-model-python/

### validation

here i am using dummy validation . pls download all the data and do correct validation

In [17]:
from keras.optimizers import Adam

In [18]:
optimizer = Adam(lr=0.001)
epochs = 100 # loaded train
file = 'weights//dense_seperable_sigmoid_15//weights.{epoch:02d}-{val_loss:.2f}.hdf5'
ckpt_save =keras.callbacks.ModelCheckpoint(file, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
ckpt_tensorboard = keras.callbacks.TensorBoard(log_dir='./dense_seperable_sigmoid_15', histogram_freq=0, batch_size=32, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

ckpt_list = [ckpt_save,ckpt_tensorboard]
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=['accuracy'])

model.fit_generator(training_generator,epochs = epochs,callbacks = ckpt_list,validation_data = val_generator ,steps_per_epoch=len(image_path)/batch_size,validation_steps=len(image_path_val)/batch_size )

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
1875/1874 [==============================] - 1068s 569ms/step - loss: 0.2149 - acc: 0.9298 - val_loss: 0.4131 - val_acc: 0.8660
Epoch 2/100
1875/1874 [==============================] - 1030s 549ms/step - loss: 0.2141 - acc: 0.9302 - val_loss: 0.4118 - val_acc: 0.8699
Epoch 3/100
1875/1874 [==============================] - 1024s 546ms/step - loss: 0.2150 - acc: 0.9305 - val_loss: 0.3874 - val_acc: 0.8724
Epoch 4/100
1875/1874 [==============================] - 1012s 540ms/step - loss: 0.2129 - acc: 0.9307 - val_loss: 0.3909 - val_acc: 0.8718
Epoch 5/100
1875/1874 [==============================] - 894s 477ms/step - loss: 0.2113 - acc: 0.9307 - val_loss: 0.3951 - val_acc: 0.8692
Epoch 6/100
1875/1874 [==============================] - 896s 478ms/step - loss: 0.2103 - acc: 0.9309 - val_loss: 0.4639 - val_acc: 0.8628
Epoch 7/100
1875/1874 [==============================] - 897s 478ms/step - loss: 0.2093 - acc: 0.9309 - val_loss: 

1073/1874 [================>.............] - ETA: 7:46 - loss: 0.2071 - acc: 0.9299

KeyboardInterrupt: 